In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import sys
import time
import json
import numpy as np
import pandas as pd
import geopandas as gpd
import pickle as pkl
import networkx as nx
import matplotlib.pyplot as plt

from pprint import pprint

import src
from src.reload import deep_reload

In [7]:
gdf = gpd.read_file('Data/atlas/atlas.shp')

In [8]:
gdf.keys()

Index(['OBJECTID', 'ID', 'DIR', 'LENGTH', 'LINKID', 'COUNTRY', 'JURISCODE',
       'JURISNAME', 'ROADNUM', 'ROADNAME', 'ADMIN', 'SURFACE', 'LANES',
       'SPEEDLIM', 'CLASS', 'NHS', 'BORDER', 'ShapeSTLen', 'geometry'],
      dtype='object')

In [9]:
gdf.head(5)

,OBJECTID,ID,DIR,LENGTH,LINKID,COUNTRY,JURISCODE,JURISNAME,ROADNUM,ROADNAME,ADMIN,SURFACE,LANES,SPEEDLIM,CLASS,NHS,BORDER,ShapeSTLen,geometry
0,22,440322,0,0.45,02_2876875,2,02_06,California,S1,PACIFIC COAST HWY,State,Paved,4,64,3,10,0,542.754634,"LINESTRING (-13114814.819 3969167.235, -131147..."
1,23,440617,0,0.02,02_2876938,2,02_06,California,S1,PACIFIC COAST HWY,State,Paved,4,64,3,10,0,18.261974,"LINESTRING (-13114334.920 3968942.957, -131143..."
2,24,440616,0,0.70,02_2876950,2,02_06,California,S1,PACIFIC COAST HWY,State,Paved,4,64,3,10,0,839.703926,"LINESTRING (-13114316.998 3968939.618, -131142..."
3,25,440621,0,0.15,02_2877050,2,02_06,California,S1,PACIFIC COAST HWY,State,Paved,4,72,3,10,0,176.664288,"LINESTRING (-13112851.365 3968056.843, -131128..."
4,26,440622,0,0.11,02_2877079,2,02_06,California,S1,PACIFIC COAST HWY,State,Paved,4,72,3,10,0,134.527078,"LINESTRING (-13112702.197 3967964.282, -131126..."


In [10]:
gdf['ROADNUM'].unique()

array(['S1', nan, 'I5', 'S73', 'S76', 'I15', 'I215', 'S78', 'S86', 'S195',
       'S111', 'S62', 'I10', 'S74', 'S243', 'S371', 'S79', 'CR3', 'C3',
       'S60', 'I8', 'S115', 'S7', 'U115', 'S98', 'S94', 'CS80', 'S75',
       'S67', 'S905', 'I805', 'S54', 'S125', 'S163', 'S282', 'S209', 'S8',
       'S15', 'S56', 'S52', 'C11 S', 'S188', 'C80 S', 'S186', 'S8 B',
       'S91', 'S210', 'S138', 'U395', 'S18', 'S178', 'S190', 'I605',
       'I210', 'S168', 'S266', 'S4', 'S58', 'S139', 'U199', 'S299',
       'U101', 'U97', 'S96', 'S162', 'S99', 'S41', 'S140', 'S132', 'S152',
       'S33', 'I580', 'S13', 'S185', 'I880', 'S180', 'S203', 'I205', 'U6',
       'S88', 'S12', 'S84', 'S160', 'I80', 'S120', 'S108', 'S26', 'S128',
       'S121', 'S16', 'I680', 'S68', 'CG17', 'S183', 'S65', 'S70', 'S20',
       'S149', 'S191', 'S49', 'S101', 'S156', 'S129', 'U156', 'CG14',
       'S32', 'S36', 'S89', 'I280', 'S44', 'S9', 'S113', 'S45', 'S61',
       'S29', 'S17', 'S35', 'S238', 'S92', 'S130', 'S85', 'I3

In [12]:
node_attributes = {
    "distance": "lambda n: 0",
    "speed": "lambda n: 0",
    "time": "lambda n: 0",
    "price": "lambda n: 0",
}

link_attributes = {
    "distance": "lambda e: e['LENGTH']*1e3",
    "speed": "lambda e: e['SPEEDLIM']/3.6",
    "time": "lambda e: (e['LENGTH']*1e3)/(e['SPEEDLIM']/3.6)",
    "price": "lambda e: 0",
    "roadnum": "lambda e: e['ROADNUM']",
}

filepath = 'Data/atlas/atlas.shp'

atlas = src.graph.graph_from_shapefile(
    filepath,
    node_attributes = node_attributes,
    link_attributes = link_attributes,
)

In [16]:
for source, adj in atlas._adj.items():
    for target, edge in adj.items():

        atlas._node[source]['roadnum'] = edge['roadnum']
        atlas._node[target]['roadnum'] = edge['roadnum']

In [18]:
atlas._adj[0][1], atlas._node[0], atlas._node[1]

({'distance': 450.0,
  'speed': 17.77777777777778,
  'time': 25.3125,
  'price': 0,
  'roadnum': 'S1'},
 {'x': -117.812386000354,
  'y': 33.554719000086315,
  'distance': 0,
  'speed': 0,
  'time': 0,
  'price': 0,
  'roadnum': 'S1'},
 {'x': -117.80807499967855,
  'y': 33.553040000114464,
  'distance': 0,
  'speed': 0,
  'time': 0,
  'price': 0,
  'roadnum': 'S1'})

In [20]:
filename = 'Outputs/atlas.json'

src.graph.graph_to_json(atlas, filename)